In [1]:
import hashlib
from typing import List, Dict, Any
from datetime import datetime
from pymongo import MongoClient
import secrets


In [2]:
# ==============================
# CONFIGURATION
# ==============================

MONGO_URI = "YOUR_MONGODB_ATLAS_CONNECTION_STRING"
DATABASE_NAME = "merkle_zkp_db"
COLLECTION_NAME = "verifications"

# Attempt to connect to MongoDB Atlas
# If connection fails, use in-memory fallback
client = None
collection = None
use_fallback = False
fallback_data = []

try:
    client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
    client.admin.command('ping')  # Test connection
    db = client[DATABASE_NAME]
    collection = db[COLLECTION_NAME]
    print("Connected to MongoDB Atlas ✅")
except Exception as e:
    print(f"⚠️ MongoDB connection failed: {str(e)[:100]}")
    print("Using in-memory fallback database for demo purposes")
    use_fallback = True
    print("To use MongoDB Atlas, update MONGO_URI with your connection string from https://www.mongodb.com/cloud/atlas")


Connected to MongoDB Atlas ✅


In [3]:
def sha256(data: bytes) -> str:
    return hashlib.sha256(data).hexdigest()

def hash_string(value: str) -> str:
    return sha256(value.encode("utf-8"))


In [4]:
def generate_merkle_root(leaves: List[str]) -> str:
    if not leaves:
        raise ValueError("Leaf list cannot be empty")

    level = leaves[:]
    while len(level) > 1:
        next_level = []
        for i in range(0, len(level), 2):
            left = level[i]
            right = level[i + 1] if i + 1 < len(level) else left
            combined_hash = sha256((left + right).encode())
            next_level.append(combined_hash)
        level = next_level
    return level[0]

def verify_merkle_root(leaves: List[str], expected_root: str) -> bool:
    return generate_merkle_root(leaves) == expected_root


In [5]:
# ==============================
# GENERIC ZERO-KNOWLEDGE PROOF
# ==============================

# This ZKP is simplified for demonstration.
# It proves that you "know a secret" (merkle_root) without revealing it.

def generate_zkp(secret: str) -> Dict[str, str]:
    """
    Generate a dummy zero-knowledge proof for a secret
    """
    nonce = secrets.token_hex(16)
    commitment = sha256((secret + nonce).encode())
    proof = {
        "commitment": commitment,
        "nonce": nonce
    }
    return proof

def verify_zkp(secret: str, proof: Dict[str, str]) -> bool:
    """
    Verify dummy zero-knowledge proof
    """
    expected_commitment = sha256((secret + proof["nonce"]).encode())
    return expected_commitment == proof["commitment"]


In [ ]:
def store_record(
    user_id: str,
    merkle_root: str,
    file_count: int,
    zkp_proof: Dict[str, str]
) -> Dict[str, Any]:

    record = {
        "user_id": user_id,
        "merkle_root": merkle_root,
        "file_count": file_count,
        "zkp_proof": zkp_proof,
        "timestamp": datetime.utcnow()
    }

    if use_fallback:
        fallback_data.append(record)
    else:
        collection.insert_one(record)
    
    return record

def get_user_history(user_id: str):
    if use_fallback:
        return [r for r in fallback_data if r["user_id"] == user_id]
    else:
        return list(collection.find({"user_id": user_id}, {"_id": 0}))


In [7]:
# 1️⃣ Simulated file hashing
files = ["file1.pdf", "file2.pdf", "file3.pdf"]
file_hashes = [hash_string(f) for f in files]

print("File Hashes:")
print(file_hashes)

# 2️⃣ Generate Merkle Root
merkle_root = generate_merkle_root(file_hashes)
print("\nMerkle Root:")
print(merkle_root)

# 3️⃣ Generate ZKP
zkp_proof = generate_zkp(merkle_root)

# 4️⃣ Verify ZKP before storing
if verify_zkp(merkle_root, zkp_proof):
    print("\nZKP Verified ✅")
else:
    raise Exception("ZKP Failed ❌")

# 5️⃣ Store in MongoDB Atlas
record = store_record(
    user_id="user_123",
    merkle_root=merkle_root,
    file_count=len(file_hashes),
    zkp_proof=zkp_proof
)

print("\nStored Record:")
print(record)

# 6️⃣ Verify Merkle Root
print("\nMerkle Root Valid:", verify_merkle_root(file_hashes, merkle_root))

# 7️⃣ Fetch History
history = get_user_history("user_123")
print("\nUser History:")
print(history)


File Hashes:
['2bc7f5da08ebda7d761bbfd4bed3c6c2718492aa4e0e2063786219bc7687a40c', 'aa78d5a5c01ef76017576055d20f3007093aa9ba583655f0b236f7506e6bfd60', 'fe3a0126360ed6fcd6bec4f663d404d9791f1b529c7d239024451d57bfd8333a']

Merkle Root:
0a75c57d690d637366bbf3706259f0393dbed6c86eab9e53a56b30062eebd283

ZKP Verified ✅


C:\Users\USER\AppData\Local\Temp\ipykernel_18644\1886946449.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow()


ServerSelectionTimeoutError: your_mongodb_atlas_connection_string:27017: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 69971fb0cdc7b79fa1ab9a8c, topology_type: Unknown, servers: [<ServerDescription ('your_mongodb_atlas_connection_string', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('your_mongodb_atlas_connection_string:27017: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>